# Results handling

This worked example shows how to work with Breathe Design results objects, from quick equilibrium KPIs to dynamic simulations and batch runs.

You’ll learn to master:
- Fetch equilibrium KPIs for a baseline cell
- Inspect and compare designs
- Run a worked example simulation with a `Cycler`
- Explore dynamic data and plot key variables
- Scale up to batch simulations and summarize results


## Setup  
<a id="setup"></a>

This imports the Breathe Design API and cycler package used throughout the worked example


In [1]:
from breathe_design import api_interface as api, Cycler

The first result we can get from the api is the equilibrium KPIs


## Designs & Equilibrium KPIs  

We start with a **baseline cell** (here: `"Molicel P45B"`).  
`get_eqm_kpis` returns a results object with convenient accessors for capacity and other KPIs.



In [2]:
eqm_results = api.get_eqm_kpis("Molicel P45B")

The results class is


In [3]:
eqm_results.__class__

breathe_design.results.SingleSimulationResults

With methods and attributes


In [4]:
for attr in dir(eqm_results):
    if not attr.startswith("_"):
        print(f"{attr}")

baseline
capacity
compare_designs
design_names
dynamic_data
dynamic_kpis
get_dynamic_data
get_input_parameters
get_kpis
get_normalized_kpis
get_summary
kpis
plot_dynamic_response
plot_radar
plot_sensitivities
plot_voltage_response
print_dynamic_variables


`baseline` provides a shortcut to the Baseline KPIs


In [41]:
eqm_results.baseline

KPI
Capacity [Ah]                           4.500000
Nominal Voltage [V]                     3.656630
Energy [Wh]                            16.454837
Gravimetric Energy Density [Wh/kg]    241.196608
Volumetric Energy Density [Wh/l]      665.310409
Minimum Anode Voltage [mV]             80.490746
Weight [g]                             68.221675
Volume [l]                              0.024733
Heat Capacity [kJ/K]                    0.052220
Name: Baseline, dtype: float64

`capacity` gets the capacity value for the Baseline design


In [42]:
eqm_results.capacity

np.float64(4.499999999999997)

`compare_designs` compares the KPIs of the designs but will return an error if only the Baseline design is present


In [43]:
try:
    eqm_results.compare_designs()
except ValueError as e:
    print(e)

Only Baseline design found. Cannot compare designs.


`design_names` gets list of design names.


In [44]:
eqm_results.design_names

['Baseline']

`dynamic_data` returns the dynamic data as result of passing a cycler to the api and running a simulation for each design and input parameter. However, in this case it will be empty as no designs or cycler objects were passed.


In [45]:
eqm_results.dynamic_data

{}

`get_input_parameters` gets the original input parameters used for the api call.


In [46]:
eqm_results.get_input_parameters()

{'base_battery': 'Molicel P45B', 'designs': [], 'formats': []}

`get_kpis` and `kpis` both return the kpi data. `get_kpi` can accept a design name and only return data for that design providing it exists.


In [47]:
try:
    eqm_results.get_kpis("testDesign")
except ValueError as e:
    print(e)

Design testDesign not found. Cannot normalize results.


`get_normalized_kpis` divides the values for each design by the Baseline values. If there is only a Baseline design this will return all 1.0s.


In [48]:
eqm_results.get_normalized_kpis()

,Baseline
KPI,
Capacity [Ah],1.0
Nominal Voltage [V],1.0
Energy [Wh],1.0
Gravimetric Energy Density [Wh/kg],1.0
Volumetric Energy Density [Wh/l],1.0
Minimum Anode Voltage [mV],1.0
Weight [g],1.0
Volume [l],1.0
Heat Capacity [kJ/K],1.0


`get_summary` prints a summary of the results


In [49]:
eqm_results.get_summary()

{'is_batch': False,
 'num_simulations': 1,
 'design_names': ['Baseline'],
 'num_designs': 1,
 'input_parameters': {'base_battery': 'Molicel P45B',
  'designs': [],
  'formats': []}}

`plot_sensitivities` plots the sensitivities of each KPI to variation in each design parameter


In [50]:
eqm_results.plot_sensitivities()

Some of the methods will not work without designs or dynamic data generated from passing a cycler to the api


In [51]:
try:
    eqm_results.plot_radar()
except ValueError as e:
    print(e)

Only Baseline design found. Cannot compare designs.


In [52]:
try:
    eqm_results.plot_dynamic_response()
except ValueError as e:
    print(e)

No dynamic data available for this simulation


In [53]:
try:
    eqm_results.plot_voltage_response()
except ValueError as e:
    print(e)

No dynamic data available for this simulation


In [54]:
try:
    eqm_results.print_dynamic_variables()
except ValueError as e:
    print(e)

No dynamic data available for this simulation


## Designs & Dynamic Data  
<a id="designs--dynamic-data"></a>

Lets add a virtual cell design and run a dynamic simulation in parallel with our baseline cell. In this example we will adjust the NPratio to a fixed value of 1.0.


In [55]:
designs = [{"designName": "Lower NP", "NPratio": 1.0}]

### Build a Cycler

A `Cycler` describes the protocol (e.g., CC charge to a voltage limit).  
Here we set units and (optionally) pass in the baseline capacity for relative C‑rates.


In [56]:
cycler = Cycler(selected_unit="C", cell_capacity=eqm_results.capacity)

### Worked Example: P45B Charge Run  
<a id="worked-example-p45b-charge-run"></a>

We’ll run **one** simulation using our cycler and a simple design list (including Baseline).  
This returns a results object with:
- `.kpis` and `.get_kpis(designName)` for design‑specific KPIs  
- `.compare_designs()` to compare across designs  
- `.plot_*` helpers for quick visualizations


In [57]:
single_output = api.run_sim(
    base_battery="Molicel P45B",
    cycler=cycler.cccv(1.0, -1.0, 0.01, 4.2, 2.6, 60.0, 60.0),
    designs=designs,
    initialSoC=0.5,
    initialTemperature_degC=25.0,
    ambientTemperature_degC=25.0,
)

In [58]:
single_output.kpis

,Baseline,Lower NP
KPI,,
Capacity [Ah],4.500000,4.626559
Nominal Voltage [V],3.656630,3.659476
Energy [Wh],16.454837,16.930780
Gravimetric Energy Density [Wh/kg],241.196608,246.933081
Volumetric Energy Density [Wh/l],665.310409,684.554009
Minimum Anode Voltage [mV],80.490746,77.407029
Weight [g],68.221675,68.564246
Volume [l],0.024733,0.024733
Heat Capacity [kJ/K],0.052220,0.052487


If we only want the equilibrium KPIs for a specific design, we can use `.get_kpis(designName)`

In [59]:
single_output.get_kpis("Lower NP")

,Lower NP
KPI,
Capacity [Ah],4.626559
Nominal Voltage [V],3.659476
Energy [Wh],16.930780
Gravimetric Energy Density [Wh/kg],246.933081
Volumetric Energy Density [Wh/l],684.554009
Minimum Anode Voltage [mV],77.407029
Weight [g],68.564246
Volume [l],0.024733
Heat Capacity [kJ/K],0.052487


We can now compare the results of our virtual cell design with the baseline cell.

In [60]:
single_output.compare_designs()

In [61]:
single_output.plot_voltage_response()

In [62]:
single_output.print_dynamic_variables()

Available variables:
Time
voltModel
voltAnode
voltCathode
ocvModel
ocvAnode
ocvCathode
tempSurfaceModel
socModel
socAnode
socCathode
currModel
heatFluxModel


In [63]:
single_output.plot_dynamic_response("tempSurfaceModel")

## Batch Results  
<a id="batch-results"></a>

Now we scale up: pass **lists** (e.g., `initialSoC`) to evaluate multiple starting conditions in one call.

- Use `.get_summary()` for an at‑a‑glance table across runs  
- Use `.get_dynamic_data(design)` to retrieve time‑series data  
- Use `.plot_dynamic_response(variable)` to visualize across designs/runs


In [64]:
initial_socs = [0.1, 0.25, 0.5, 0.75, 0.9]
batch_output = api.run_sim(
    base_battery="Molicel P45B",
    cycler=cycler.cc_chg(1.0, 4.2),
    designs=designs,
    initialSoC=initial_socs,  # List of SOC values
    initialTemperature_degC=25.0,
    ambientTemperature_degC=25.0,
)

Here we get a different class but the methods should be the very similar


In [65]:
batch_output.__class__

breathe_design.results.BatchSimulationResults

In [66]:
for attr in dir(batch_output):
    if not attr.startswith("_"):
        print(f"{attr}")

baseline
capacity
compare_designs
design_names
dynamic_data
dynamic_kpis
first
get_dynamic_data
get_input_parameters
get_kpis
get_normalized_kpis
get_summary
num_simulations
plot_dynamic_response
plot_radar
plot_sensitivities
plot_voltage_response
print_dynamic_variables


In [67]:
batch_output.get_summary()

{'is_batch': True,
 'num_simulations': 5,
 'design_names': ['Baseline', 'Lower NP'],
 'num_designs': 2,
 'input_parameters': [{'base_battery': 'Molicel P45B',
   'cycler': {'cycle_type': 'CC_CHG',
    'selected_unit': 'C',
    'control_parameters': {'I_chg': 1.0, 'V_max': 4.2}},
   'designs': [{'designName': 'Lower NP', 'NPratio': 1.0}],
   'initialSoC': 0.1,
   'initialTemperature_degC': 25.0,
   'ambientTemperature_degC': 25.0},
  {'base_battery': 'Molicel P45B',
   'cycler': {'cycle_type': 'CC_CHG',
    'selected_unit': 'C',
    'control_parameters': {'I_chg': 1.0, 'V_max': 4.2}},
   'designs': [{'designName': 'Lower NP', 'NPratio': 1.0}],
   'initialSoC': 0.25,
   'initialTemperature_degC': 25.0,
   'ambientTemperature_degC': 25.0},
  {'base_battery': 'Molicel P45B',
   'cycler': {'cycle_type': 'CC_CHG',
    'selected_unit': 'C',
    'control_parameters': {'I_chg': 1.0, 'V_max': 4.2}},
   'designs': [{'designName': 'Lower NP', 'NPratio': 1.0}],
   'initialSoC': 0.5,
   'initialTemp

In [68]:
batch_output.get_kpis()

,Baseline,Lower NP
KPI,,
Capacity [Ah],4.500000,4.626559
Nominal Voltage [V],3.656630,3.659476
Energy [Wh],16.454837,16.930780
Gravimetric Energy Density [Wh/kg],241.196608,246.933081
Volumetric Energy Density [Wh/l],665.310409,684.554009
Minimum Anode Voltage [mV],80.490746,77.407029
Weight [g],68.221675,68.564246
Volume [l],0.024733,0.024733
Heat Capacity [kJ/K],0.052220,0.052487


We can plot the variables of interest for the designs across the batch

In [69]:
batch_output.plot_dynamic_response("voltModel")

In [70]:
batch_output.plot_dynamic_response("tempSurfaceModel")

We can also retrieve all of the dynamic data for the batch for all designs, or for a single design


In [71]:
batch_output.dynamic_data

[{'Baseline': {'Time': [0,
    5.6218269514104555e-09,
    5.6218269514104554e-08,
    5.115862525783514e-07,
    4.609898100156573e-06,
    4.559301657593879e-05,
    0.0004554242013337609,
    0.0017231747057562892,
    0.003928865814958606,
    0.007481532612260344,
    0.014308295900367671,
    0.029243109233920907,
    0.06080681194829878,
    0.12335651995953212,
    0.2451179040012637,
    0.4866763882692177,
    0.9740587099813205,
    1.6666990472278949,
    2.3593393844744694,
    3.051979721721044,
    3.7446200589676186,
    4.437260396214193,
    5.129900733460768,
    5.822541070707342,
    6.666743702525612,
    7.510946334343882,
    8.355148966162151,
    9.400515651334981,
    10.445882336507811,
    11.491249021680641,
    12.845682338138916,
    14.200115654597191,
    15.903978488904134,
    17.607841323211076,
    19.810192913100707,
    22.551146981321843,
    25.866932746562078,
    29.941079713961738,
    35.22819327040568,
    42.656953376274,
    50.085713482

In [72]:
batch_output.get_dynamic_data("Baseline")

[{'Baseline': {'Time': [0,
    5.6218269514104555e-09,
    5.6218269514104554e-08,
    5.115862525783514e-07,
    4.609898100156573e-06,
    4.559301657593879e-05,
    0.0004554242013337609,
    0.0017231747057562892,
    0.003928865814958606,
    0.007481532612260344,
    0.014308295900367671,
    0.029243109233920907,
    0.06080681194829878,
    0.12335651995953212,
    0.2451179040012637,
    0.4866763882692177,
    0.9740587099813205,
    1.6666990472278949,
    2.3593393844744694,
    3.051979721721044,
    3.7446200589676186,
    4.437260396214193,
    5.129900733460768,
    5.822541070707342,
    6.666743702525612,
    7.510946334343882,
    8.355148966162151,
    9.400515651334981,
    10.445882336507811,
    11.491249021680641,
    12.845682338138916,
    14.200115654597191,
    15.903978488904134,
    17.607841323211076,
    19.810192913100707,
    22.551146981321843,
    25.866932746562078,
    29.941079713961738,
    35.22819327040568,
    42.656953376274,
    50.085713482